<img src="../logo.png" height="200" width="900"> 

# <center> Сбор данных: грязная работа своими руками </center>

# 1. Как выглядит хранилище мемов 

## 1.1. Что мы хотим получить


Итак, мы хотим  написать код, который поможет нам скачать полезные данные с сайта [knowyourmeme.com:](http://knowyourmeme.com)

- **Name** – название мема,
- **Origin_year** – год его создания,
- **Views** – число просмотров,
- **About** – текстовое описание мема,
- ** и многие другие** 

После скачивания и чистки данных, можно будет заняться оцениванием моделей. Например, можно попробовать предсказать популярность мема по его параметрам. Но это все позже, а сейчас познакомимся с парой определений:

* **Парсер** — это скрипт, который собирает информацию с сайта
* **Краулер** — это часть парсера, которая переходит по ссылкам
* **Краулинг** — это переход по страницам и ссылкам
* **Скрапинг** — это сбор данных со страниц
* **Парсинг** — это сразу и краулинг и скрапинг

## 1.2.  Что такое HTML 

**HTML (HyperText Markup Language)**  — это язык разметки. Он является стандартным для написания сайтов. Команды в таком языке называются **тегами**. Если открыть абсолютно любой сайт, нажать на правую кнопку мышки, а после нажать `View page source`, то перед вами предстанет HTML скелет этого сайта. 

Можно увидеть, что HTML-страница это ни что иное, как набор вложенных тегов. Можно заметить, например, следующие теги:

- `<title>` – заголовок страницы
- `<h1>…<h6>` – заголовки разных уровней
- `<p>` – абзац (paragraph)
- `<div>` – выделения фрагмента документа с целью изменения вида содержимого
- `<table>` – прорисовка таблицы 
- `<tr>` – разделитель для строк в таблице 
- `<td>` – разделитель для столбцов в таблице
- `<b>` – устанавливает жирное начертание шрифта

Обычно команда `<...>` открывает тег, а  `</...>` закрывает его. Все, что находится между этими двумя командами, подчиняется правилу, которое диктует тег. Например, все, что находится между `<p>` и  `</p>` — это отдельный абзац.   

Теги образуют своеобразное дерево с корнем в теге `<html>` и разбивают страницу на разные логические кусочки. У каждого тега могут быть свои потомки (дети) — те теги, которые вложены в него, и свои родители. 

Например, HTML-древо страницы может выглядеть вот так:

    <html>
    <head> Заголовок </head>
    <body>
        <div> 
            Первый кусок текста со своими свойствами
        </div>
        <div>
            Второй кусок текста
                <b>
                    Третий кусок с выделенным текстом
                </b>
        </div>
        Четвёртый кусок текста        
    </body>
    </html>            
    
    
<img align="center" src="pictures/tree.png" height="500" width="500"> 

Можно работать с этим html как с текстом, а можно как с деревом. Обход этого дерева и есть парсинг веб-страницы. Нам нужно находить нужные нам узлы среди всего этого разнообразия и забирать с них информацию.

Вручную обходить эти деревья неудобно, поэтому есть специальные языки для обхода деревьев.

- [CSS-селектор](https://ru.wikibooks.org/wiki/CSS/%D0%A1%D0%B5%D0%BB%D0%B5%D0%BA%D1%82%D0%BE%D1%80%D1%8B) (это когда мы ищем элемент страницы по паре ключ, значение)
- [XPath](https://ru.wikipedia.org/wiki/XPath) (это когда мы прописываем путь по дереву вот так: /html/body/div[1]/div[3]/div/div[2]/div)
- Различные библиотеки, например, BeautifulSoup для питона. Именно эту библиотеку мы и будем использовать. 

## 1.3. Наш первый запрос

Доступ к веб-станицам позволяет получать модуль `requests`. Подгрузим его и ещё пару пакетов.

In [1]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек 
import time          # Библиотека для времени

Для наших благородных исследовательских целей нужно собрать данные по каждому мему с соответствующей ему страницы. Но для начала нужно получить адреса этих страниц. Поэтому открываем основную страницу со всеми выложенными мемами. Выглядит она следующим образом:

<img align="center" src="pictures/memes_main.png" height="500" width="500"> 

Отсюда мы и будем собирать ссылки. Сохраним в переменную `page_link` адрес основной страницы и откроем её при помощи библиотеки `requests`.

In [2]:
page_link = 'http://knowyourmeme.com/memes/all/page/1'

In [3]:
response = requests.get(page_link)
response

<Response [403]>

А вот и первая проблема! Обращаемся к [главному источнику знаний](https://en.wikipedia.org/wiki/HTTP_403) и выясняем, что 403-я ошибка выдается сервером, если он доступен и способен обрабатывать запросы, но по некоторым личным причинам отказывается это делать. 

Попробуем выяснить, почему. Для этого проверим, как выглядел финальный запрос, отправленный нами на сервер.

In [4]:
for key, value in response.request.headers.items():
    print(key+": "+value)

User-Agent: python-requests/2.21.0
Accept-Encoding: gzip, deflate
Accept: */*
Connection: keep-alive


Похоже, мы недвусмысленно дали понять серверу, что мы используем python, а именно библиотеку requests версии 2.14.2. Скорее всего, это вызвало у сервера некоторые подозрения относительно наших благих намерений и он решил нас безжалостно отвергнуть. Для сравнения, можно посмотреть, как выглядят request-headers у запроса через браузер:

<img align="center" src="pictures/good_headers.png" height="800" width="800"> 

Очевидно, что нашему скромному запросу не тягаться с таким обилием мета-информации, которое передается при запросе из обычного браузера. К счастью, никто нам не мешает притвориться человечными.

Библиотек, которые справляются с такой задачей, существует очень много, мы воспользуемся [`fake-useragent`](https://pypi.python.org/pypi/fake-useragent). При вызове метода из различных кусочков будет генерироваться случайное сочетание операционной системы, спецификаций и версии браузера, которые можно передавать в запрос:

In [5]:
from fake_useragent import UserAgent

In [6]:
UserAgent().chrome

'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/44.0.2403.155 Safari/537.36'

In [7]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

Cоединение установлено и данные получены.

In [8]:
html = response.content
html[:1000]

b'<!DOCTYPE html>\n<html xmlns:fb=\'https://www.facebook.com/2008/fbml\' xmlns=\'https://www.w3.org/1999/xhtml\'>\n<head>\n<meta content=\'text/html; charset=utf-8\' http-equiv=\'Content-Type\'>\n<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHlcJWg==","queueTime":0,"applicationTime":54,"agent":""}</script>\n<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"c1a6d52f38",applicationID:"31165848"};window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var i=n[t]={exports:{}};e[t][0].call(i.exports,function(n){var i=e[t][1][n];return r(i||n)},i,i.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<t.length;i++)r(t[i]);return r}({1:[function(e,n,t){function r(){}function i(e,n,t){return function(){return o(e,[u.now(

## 1.4. Beautiful Soup

<img align="center" src="pictures/soup.jpg" height="200" width="200"> 

Пакет **[bs4 , a.k.a BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/)** был назван в честь стиха про красивый суп из Алисы в стране чудес.

BeautifulSoup — это библиотека, которая из необработанного HTML кода страницы создаёт структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием `BeautifulSoup` — скорее всего, не то, что нам нужно. Это третья версия (*Beautiful Soup 3*), а мы будем использовать четвертую. Нужно будет установить пакет `beautifulsoup4`. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — `bs4`, а импортировать функцию под названием `BeautifulSoup`. В общем, сначала легко запутаться, но эти трудности нужно преодолеть.

```
pip install beautifulsoup4
```

In [9]:
from bs4 import BeautifulSoup

Передадим функции `BeautifulSoup` текст веб-страницы, которую мы скачали выше.

In [10]:
soup = BeautifulSoup(html,'html.parser')

Посмотрим что лежит внутри переменной `soup`. Невнимательный пользователь, скорее всего, скажет,что ничего вообще не изменилось. Тем не менее, это не так. Теперь мы можем свободно перемещаться по HTML-дереву страницы и искать нужные нам теги.

In [13]:
soup.html.head.title

<title>All Entries | Know Your Meme</title>

Можно извлечь из того места, где мы оказались, текст с помощью метода `text`.

In [14]:
soup.html.head.title.text

'All Entries | Know Your Meme'

Более того, зная адрес элемента, мы сразу можем найти его. Например, можно сделать это по классу. Следующая команда должна найти элемент, который лежит внутри тега `a` и имеет класс `photo`.

In [15]:
obj = soup.find('a', attrs = {'class':'photo'})
obj

<a class="photo left" href="/memes/berniebruh" target="_self"><img alt="Screenshot of three participants in the #BernieBruh movement. This article explains the #berniebruh hashtag campaign. " data-src="https://i.kym-cdn.com/featured_items/icons/wide/000/010/485/53b.jpg" height="112" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title='#BernieBruh Hashtag Fights Back Against "Bernie Bro" Stereotypes' width="198"/> <div class="info abs"> <div class="c"> #BernieBruh Hashtag Fights Back Against "Bernie Bro" Stereotypes </div> </div> </a>

Однако, вопреки нашим ожиданиям, извлечённый объект имеет класс `"photo left"`. Оказывается, `BeautifulSoup4` расценивает аттрибуты `class` как набор отдельных значений, поэтому `"photo left"` для библиотеки равносильно `["photo", "left"]`, а указанное нами значение этого класса `"photo"` входит в этот список. Чтобы избежать такой неприятной ситуации, придется воспользоваться собственной функцией и задать строгое соответствие:

In [16]:
obj = soup.find(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
obj

<a class="photo" href="/memes/people/jreg"><img alt="Jreg" data-src="https://i.kym-cdn.com/entries/icons/medium/000/033/034/jreg.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Jreg"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> <span class="label" style="background: #d32f2e; color: white;">Person</span> </div> </a>

Полученный после поиска объект также обладает структурой bs4. Поэтому можно продолжить искать нужные нам объекты уже в нём. Извлечём ссылку на этот мем. Сделать это можно по атрибуту `href`, в котором лежит наша ссылка. 

In [17]:
obj.attrs['href']

'/memes/people/jreg'

In [ ]:
obj.get('href')

Обратите внимание, что после преобразований у данных поменялся тип. Теперь он `str`. Это означет, что с ними можно работать как с текстом. 

In [18]:
print("Тип данных до вытаскивания ссылки:", type(obj))
print("Тип данных после вытаскивания ссылки:", type(obj.attrs['href']))

Тип данных до вытаскивания ссылки: <class 'bs4.element.Tag'>
Тип данных после вытаскивания ссылки: <class 'str'>


Если несколько элементов на странице обладают указанным адресом, то метод `find` вернёт только самый первый.  Чтобы найти все элементы с таким адресом, нужно использовать метод `findAll`, и на выход будет выдан список. Таким образом, мы можем получить одним поиском сразу все объекты, содержащие ссылки на страницы с мемами.

In [19]:
meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
meme_links[:3]

[<a class="photo" href="/memes/people/jreg"><img alt="Jreg" data-src="https://i.kym-cdn.com/entries/icons/medium/000/033/034/jreg.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Jreg"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> <span class="label" style="background: #d32f2e; color: white;">Person</span> </div> </a>,
 <a class="photo" href="/memes/sometimes-maybe-good-sometimes-maybe-shit"><img alt="Sometimes maybe good, sometimes maybe shit" data-src="https://i.kym-cdn.com/entries/icons/medium/000/033/033/gattuso.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Sometimes maybe good, sometimes maybe shit"/> <div class="entry-labels"> <span class="label label-deadpool"> Deadpool </span> </div> </a>,
 <a class="photo" href="/memes/events/indonesian-child-rights-official-statement-about-woman-can-get-pregnant-in-swimming-pool"><img alt="Indonesian Child Rights Offici

Осталось очистить полученный список от лишнего:

In [20]:
meme_links = [link.attrs['href'] for link in meme_links]

In [21]:
meme_links[:10]

['/memes/people/jreg',
 '/memes/sometimes-maybe-good-sometimes-maybe-shit',
 '/memes/events/indonesian-child-rights-official-statement-about-woman-can-get-pregnant-in-swimming-pool',
 '/memes/scottish-twitter-reactions-to-the-2019-20-wuhan-coronavirus-outbreak',
 '/memes/subcultures/baldurs-gate',
 '/memes/girls-in-class-looking-back',
 '/memes/people/liam-allen-miller',
 '/memes/dave-chappelle-reading-white-people-magazine',
 '/memes/the-smeeze',
 '/memes/observe-my-superior-strategic-mind-at-work']

In [22]:
len(meme_links)

16

Готово, получили ровно 16 ссылок по числу мемов на одной странице поиска. 

Чтобы легче было искать адрес элемента, можно установить для своего браузера специальную утилиту, позволяющую извлекать со страницы нужные теги, например, [selectorgadget.](http://selectorgadget.com/)

Тем не менее, этот путь не подходит для истинного самурая. Есть другой способ — искать теги для каждого нужного нам элемента вручную. Для этого придётся жать правой кнопкой мышки по окну браузера и жать кнопку **Исследовать элемент (Inspect)**. После браузер будет выглядеть так: 

<img align="center" src="pictures/memes_inspection.png" height="800" width="800"> 

Полученный html, в котором находится адрес выбранного вами объекта, можно смело копировать в код.

# 2. Собираем ссылки

После того, как мы скачали все ссылки с текущей страницы, нам нужно каким-то образом перейти на соседнюю и начать скачивать ссылки с неё. На сайте это можно сделать пролистав страницу с мемами вниз, javascript-функции подгрузят новые мемы.

Обычно, все параметры, которые мы устанавливаем на сайте для поиска, отражаются на структуре ссылки. Если мы хотим получить первую страницу с мемами, мы должны будем обратиться к сайту по ссылке 

                `http://knowyourmeme.com/memes/all/page/1`


Если мы захотим получить вторую страницу, нам придётся заменить номер страницы на 2


                `http://knowyourmeme.com/memes/all/page/2`
 
Таким образом мы сможем пройтись по всем страницам.

In [23]:
def getPageLinks(page_number):
    """
        Возвращает список ссылок на мемы, полученный с текущей страницы
        
        page_number: int/string
            номер страницы для парсинга
            
    """
    # составляем ссылку на страницу поиска
    page_link = 'http://knowyourmeme.com/memes/all/page/{}'.format(page_number)
    
    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return [] 
    
    # получаем содержимое страницы
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    # ищем ссылки на мемы и очищаем их от ненужных тэгов
    meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
    meme_links = ['http://knowyourmeme.com' + link.attrs['href'] for link in meme_links]
    
    return meme_links

In [24]:
meme_links = getPageLinks(1)
meme_links[:2]

['http://knowyourmeme.com/memes/people/jreg',
 'http://knowyourmeme.com/memes/sometimes-maybe-good-sometimes-maybe-shit']

In [25]:
meme_links = getPageLinks(2)
meme_links[:2]

['http://knowyourmeme.com/memes/why-must-you-hurt-me-in-this-way',
 'http://knowyourmeme.com/memes/taj-mahal-comparison-with-slums']

Отлично, функция работает и теперь мы теоретически можем достать все $18000$ ссылок, для чего нам нужно будет пройтись по $\frac{18000}{16} \approx 1125$ страницам. Прежде чем расстраивать сервер таким количеством запросов, посмотрим, как доставать всю необходимую информацию о конкретном меме. 

# 3. Скачиваем информацию об одном меме

По аналогии со ссылками можно извдечь что угодно. Для этого надо сделать несколько шагов: 

1. Открываем страничку с мемом
2. Находим любым способом тег для нужной нам информации
3. Вызываем Beautiful Soap
4. ......
5. Profit 

Извлечём число просмотров мема.

<img align="center" src="pictures/doge_main.png" height="600" width="600"> 

In [26]:
meme_page = 'http://knowyourmeme.com/memes/doge'

response = requests.get(meme_page, headers={'User-Agent': UserAgent().chrome})

html = response.content
soup = BeautifulSoup(html,'html.parser')

Посмотрим, как можно извлечь статистику просмотров, комментариев, а также числа загруженных видео и фото, связанных с нашим мемом. Всё это хранится справа вверху под тэгами `"dd"` и с классами  `"views"`, `"videos"`, `"photos"` и `"comments"`

In [27]:
views = soup.find('dd', attrs={'class':'views'})
views

<dd class="views" title="13,176,671 Views">
<a href="/memes/doge" rel="nofollow">13,176,671</a>
</dd>

In [28]:
views = views.find('a').text
views

'13,176,671'

In [29]:
views = int(views.replace(',', ''))
views

13176671

Напишем функцию для сбора этой статистики. 

In [30]:
def getStats(soup, stats):
    """
        Возвращает очищенное число просмотров/коментариев/...
        
        soup: объект bs4.BeautifulSoup 
            представление текущей страницы
            
        stats: string
            views/videos/photos/comments
            
    """
    try:
        obj = soup.find('dd', attrs={'class':stats})
        obj = obj.find('a').text
        obj = int(obj.replace(',', ''))
    except:
        obj=None
    
    return obj

In [31]:
views = getStats(soup, stats='views')
videos = getStats(soup, stats='videos')
photos = getStats(soup, stats='photos')
comments = getStats(soup, stats='comments')

print("Просмотры: {}\nВидео: {}\nФото: {}\nКомментарии: {}".format(views, videos, photos, comments))

Просмотры: 13176671
Видео: 64
Фото: 1698
Комментарии: 918


Еще из интересного и исследовательского —  достанем дату и время добавления мема. Если посмотреть на страницу в браузере, можно подумать, что максимум информации, который мы можем извлечь - это число лет, прошедших с момента публикации —  `Added 4 years ago by NovaXP`. Однако мы так просто сдаваться не будем, посмотрим что в html-коде страницы отвечает за эту надпись:

<img align="center" src="pictures/html_time_ago.png" height="600" width="600"> 

Ага! Вот и подробности по дате добавления, с точностью до минуты.

In [32]:
date = soup.find('abbr', attrs={'class':'timeago'}).attrs['title']
date

'2020-02-03T05:33:03-05:00'

На самом деле, парсеры — дело непредсказуемое. Часто страницы, которые мы парсим, имеют очень неоднородну структуру. Например, если мы парсим мемы, на части страниц может быть указано описание, а на части нет.

Как только код впервые встречается с отсутствием описания, он выдаёт ошибку и останавливается. Чтобы нормально собрать все данные, приходится [прописывать исключения.](https://pythonworld.ru/tipy-dannyx-v-python/isklyucheniya-v-python-konstrukciya-try-except-dlya-obrabotki-isklyuchenij.html) Для этого используют  конструкцию `try - except` 

Например, мы хотим извлечь статус мема, для этого найдем окружающие его тэги:

In [34]:
properties = soup.find('aside', attrs={'class':'left'})

In [35]:
meme_status

<dd>
Confirmed
</dd>

Дальше нужно извлечь из тэгов текст и убрать лишние пробелы.

In [36]:
meme_status.text.strip()

'Confirmed'

Однако, если неожиданно выяснится, что у мема нет статуса, метод `find` вернёт пустоту. Метод `text`, в свою очередь, не сможет найти в тэгах текст и выдаст ошибку. Чтобы обезопасить себя от таких пустот, можно прописать исключение или `if - else`. Так как в текущем меме статус все-таки есть, нарочно зададим его как пустой объект, чтобы проверить, что ошибка поймается в обоих случаях

In [37]:
# Делай раз! Ищем статус мема, но не находим его
meme_status = None

# Делай два! Пытаемся извлечь его...

# ... с исключениями
try:
    print(meme_status.text.strip()) 
# Если возникает ошибка, статус не найден, выдаём пустоту.
except:
    print("Exception")
    
    
# ... с проверкой на пустой элемент
if meme_status:
    print(meme_status.text.strip())
else:
    print("Empty")

Exception
Empty


Такой код позволяет обезопасить себя от ошибок во время работы кода.

In [38]:
properties = soup.find('aside', attrs={'class':'left'})
meme_status = properties.find("dd")

meme_status = "Empty" if not meme_status else meme_status.text.strip()
print(meme_status)

Confirmed


По аналогии можно извлечь всю остальную информацию со страницы.

In [39]:
def getProperties(soup):
    """
        Возвращает список (tuple) с названием, статусом, типом, 
        годом и местом происхождения и тэгами
        
        soup: объект bs4.BeautifulSoup 
            представление текущей страницы
    
    """
    # название - идёт с самым большим заголовком h1, легко найти
    meme_name = soup.find('section', attrs={'class':'info'}).find('h1').text.strip()
    
    # достаём все данные справа от картинки 
    properties = soup.find('aside', attrs={'class':'left'})
    
    # статус идет первым - можно не уточнять класс
    meme_status = properties.find("dd")
    
    # oneliner, заменяющий try-except: если тэга нет в properties, вернётся объект NoneType,
    # у которого аттрибут text отсутствует, и в этом случае он заменится на пустую строку
    meme_status = "" if not meme_status else meme_status.text.strip()
    
    # тип мема - обладает уникальным классом
    meme_type = properties.find('a', attrs={'class':'entry-type-link'})
    meme_type = "" if not meme_type else meme_type.text 
    
    # год происхождения первоисточника можно найти после заголовка Year, 
    # находим заголовок, определяем родителя и ищем следущего ребенка - наш раздел
    meme_origin_year = properties.find(text='\nYear\n')
    meme_origin_year = "" if not meme_origin_year else meme_origin_year.parent.find_next()
    meme_origin_year = meme_origin_year.text.strip()
    
    # сам первоисточник
    meme_origin_place = properties.find('dd', attrs={'class':'entry_origin_link'})
    meme_origin_place = "" if not meme_origin_place else meme_origin_place.text.strip()
    
    # тэги, связанные с мемом
    meme_tags = properties.find('dl', attrs={'id':'entry_tags'}).find('dd')
    meme_tags = "" if not meme_tags else meme_tags.text.strip()
    
    return meme_name, meme_status, meme_type, meme_origin_year, meme_origin_place, meme_tags

In [40]:
getProperties(soup)

('Doge',
 'Confirmed',
 'Animal',
 '2013',
 'Tumblr',
 'animal, dog, shiba inu, shibe, such doge, super shibe, japanese, super, tumblr, much, very, many, comic sans, photoshop meme, such, shiba, shibe doge, doges, dogges, reddit, comic sans ms, tumblr meme, hacked, bitcoin, dogecoin, shitposting, stare, canine')

Свойства мема собрали. Теперь собираем по аналогии его текстовое описание. 

In [41]:
def getText(soup):
    """
        Возвращает текстовые описания мема
        
        soup: объект bs4.BeautifulSoup 
            представление текущей страницы
            
    """
    
    # достаём все тексты под картинкой
    body = soup.find('section', attrs={'class':'bodycopy'})
    
    # раздел about (если он есть), должен идти первым, берем его без уточнения класса
    meme_about = body.find('p')
    meme_about = "" if not meme_about else meme_about.text
    
    # раздел origin можно найти после заголовка Origin или History, 
    # находим заголовок, определяем родителя и ищем следущего ребенка - наш раздел
    meme_origin = body.find(text='Origin') or body.find(text='History')
    meme_origin = "" if not meme_origin else meme_origin.parent.find_next().text
    
    # весь остальной текст (если он есть) можно положить в одно текстовое поле
    if body.text:
        other_text = body.text.strip().split('\n')[4:]
        other_text = " ".join(other_text).strip()
    else:
        other_text = ""
        
    return meme_about, meme_origin, other_text

In [42]:
meme_about, meme_origin, other_text = getText(soup)

print("О чем мем:\n{}\n\nПроисхождение:\n{}\n\nОстальной текст:\n{}...\n"\
      .format(meme_about, meme_origin, other_text[:200]))

О чем мем:
Doge is a slang term for "dog" that is primarily associated with pictures of Shiba Inus (nicknamed "Shibe") and internal monologue captions on Tumblr. These photos may be photoshopped to change the dog's face or captioned with interior monologues in Comic Sans font. Starting in 2017, Ironic Doge formats gained prevalence over the original wholesome version.

Происхождение:
The use of the misspelled word "doge" to refer to a dog dates back to June 24th, 2005, when it was mentioned in an episode of Homestar Runner's puppet show. In the episode titled "Biz Cas Fri 1"[2], Homestar calls Strong Bad his "d-o-g-e" while trying to distract him from his work.

Остальной текст:
Identity On February 23rd, 2010, Japanese kindergarten teacher Atsuko Sato posted several photos of her rescue-adopted Shiba Inu dog Kabosu to her personal blog.[38] Among the photos included a peculi...



Наконец, создадим функцию, возвращающую всю информацию по текущему мему

In [43]:
def getMemeData(meme_page):
    """
        Запрашивает данные по странице, возвращает обработанный словарь с данными
        
        meme_page: string
            ссылка на страницу с мемом
    
    """
    
    # запрашиваем данные по ссылке
    response = requests.get(meme_page, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    # получаем содержимое страницы
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # используя ранее написанные функции парсим информацию
    views = getStats(soup=soup, stats='views')
    videos = getStats(soup=soup, stats='videos')
    photos = getStats(soup=soup, stats='photos')
    comments = getStats(soup=soup, stats='comments')

    # дата
    date = soup.find('abbr', attrs={'class':'timeago'}).attrs['title']

    # имя, статус, и т.д.
    meme_name, meme_status, meme_type, meme_origin_year, meme_origin_place, meme_tags =\
    getProperties(soup=soup)

    # текстовые поля
    meme_about, meme_origin, other_text = getText(soup=soup)

    # составляем словарь, в котором будут хранится все полученные и обработанные данные
    data_row = {"name":meme_name, "status":meme_status, 
                "type":meme_type, "origin_year":meme_origin_year, 
                "origin_place":meme_origin_place,
                "date_added":date, "views":views, 
                "videos":videos, "photos":photos, "comments":comments, "tags":meme_tags,
                "about":meme_about, "origin":meme_origin, "other_text":other_text}

    return data_row

In [44]:
data_row = getMemeData('http://knowyourmeme.com/memes/doge')

А теперь подготовим табличку, чтобы в неё записывать все данные, добавим в неё первую полученную строку.

In [45]:
final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments', 
                                 'tags', 'about', 'origin', 'other_text'])

In [46]:
final_df = final_df.append(data_row, ignore_index=True)

In [47]:
final_df

,name,status,type,origin_year,origin_place,date_added,views,videos,photos,comments,tags,about,origin,other_text
0,Doge,Confirmed,Animal,2013,Tumblr,2020-02-03T05:33:03-05:00,13176671,64,1698,918,"animal, dog, shiba inu, shibe, such doge, supe...","Doge is a slang term for ""dog"" that is primari...","The use of the misspelled word ""doge"" to refer...","Identity On February 23rd, 2010, Japanese kind..."


Ещё раз убедимся что всё работает.

In [48]:
from tqdm import tqdm_notebook

In [49]:
for meme_link in tqdm_notebook(meme_links):
    data_row = getMemeData(meme_link)
    final_df = final_df.append(data_row, ignore_index=True)

In [50]:
final_df.head()

,name,status,type,origin_year,origin_place,date_added,views,videos,photos,comments,tags,about,origin,other_text
0,Doge,Confirmed,Animal,2013,Tumblr,2020-02-03T05:33:03-05:00,13176671,64,1698,918,"animal, dog, shiba inu, shibe, such doge, supe...","Doge is a slang term for ""dog"" that is primari...","The use of the misspelled word ""doge"" to refer...","Identity On February 23rd, 2010, Japanese kind..."
1,Why Must You Hurt Me in This Way,Submission,Image Macro,2006,YouTube,2020-02-27T16:00:53-05:00,1178,2,11,4,"puppet pals, potter puppet pals, wizard angst,...","""Why Must You Hurt Me in This Way"" is a memora...","On November 2nd, 2006, YouTuber Neil Cicierega...","Spread On June 5th, 2014, Tumblr [1] user ohha..."
2,Taj Mahal Comparison With Slums,Submission,Exploitable,2019,Instagram,2020-02-28T17:30:53-05:00,374,0,12,0,"taj mahal, object labeling, social gap, jacob ...",Taj Mahal Comparison With Slums is an image ma...,"On March, 21st 2019, Instagram [1] user @jacob...","Spread On August, 28th 2019, Reddit [2] user a..."
3,Lore Olympus,Submission,Web Series,2018,Rachel Smythe,2020-02-27T15:40:19-05:00,1514,0,3,7,"rachel smythe, jim henson, fantasy, greek myth...",Lore Olympus is a web series created by Rachel...,"On March 4th, 2018, Rachel Smythe launched Lor...",“Witness what the gods do…after dark. The frie...
4,Roscoe The Cat Sees Himself,Submission,,2019,Twitter,2020-02-27T15:22:20-05:00,561,0,10,0,"mirror, object label, tweet, kitty, cute, magn...",Roscoe The Cat Sees Himself is an image macro ...,"On August 17th, 2019, Instagram user JustBeing...","Spread On February 17th, 2020, Twitter user @p..."


In [51]:
final_df.shape

(17, 14)

# 4. Итоговый цикл

Осталось написать итоговый цикл. На всякий случай обернём его в `try-except`.

In [ ]:
from tqdm import tqdm_notebook

final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments', 
                                 'tags', 'about', 'origin', 'other_text'])

for page_number in tqdm_notebook(range(1075), desc='Pages'):
    
    # собрали ссылки с текущей страницы
    meme_links = getPageLinks(page_number)  
    
    for meme_link in tqdm_notebook(meme_links, desc='Memes', leave=False):
        
        # иногда с первого раза страничка не прогружается
        for i in range(5):
            try:
                # пытаемся собрать данные
                data_row = getMemeData(meme_link)           
                final_df = final_df.append(data_row, ignore_index=True)  
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                print('AHTUNG! parsing once again:', meme_link)
                continue

# 5. Что делать, если сервер разозлился? 


* Вы решили собрать себе немного данных 
* Сервер не в восторге от ковровой бомбардировки автоматическими запросами 
* Error 403, 404, 504, $\ldots$ 
* Капча, требования зарегистрироваться
* Заботливые сообщения, что с вашего устройства обнаружен подозрительный трафик

<center>
<img src="pictures/doge.jpg" width="450"> 

### а) быть терпеливым 

* Слишком частые запросы раздражают сервер
* Ставьте между ними временные задержки 
* Сервер любит временные задержки, так как боится сломаться от перегрузок

In [52]:
import time
time.sleep(3) # и пусть весь мир подождёт 3 секунды

### б) общаться через посредников

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/proxy.jpeg" width="400"> 

Запрос работал немного подольше, ip адрес сменился. Большая часть прокси-серверов, которые вы найдёте работают плохо. Иногда запрос идёт очень долго и выгоднее сбросить его. Это можно настроить опцией `timeout`.  Например, так если сервер не будет отвечать секунду, код перестанет работать. 

In [ ]:
import requests
requests.get('http://www.google.com', timeout=1)

## в) уходить глубже 

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/tor.jpg" width="600"> 

Можно попытаться обходить злые сервера через тор. Есть несколько способов, но мы про это говорить не будем. Лучше подробно почитать [в нашей статье на Хабре.](https://habr.com/ru/company/ods/blog/346632/)

## Совместить всё? 

1. Начните с простых приемов, например с `time.sleep`
2. Пробуйте новые приёмы постепенно
3. Каждый новый приём замедляет скорость работы
4. [Разные продвинутые способы работы с библиотекой requests](http://docs.python-requests.org/en/v0.10.6/user/advanced/)

> Напоследок, хотелось бы сказать пару слов о парсинге вообще и при помощи Тора в частности. Собирать себе данные самостоятельно - это стильно, модно и в принципе интересно, можно получить наборы, которых еще никто никогда не обрабатывал, сделать что-то новое, посмотреть, наконец, на все мемы мира сразу. Однако не стоит забывать, что ограничения, введенные сервером, в том числе баны, появились не просто так, а в целях защиты сайта от DDoS-атак. К чужому труду стоит относится с уважением, и даже если у сервера никакой защиты нет, - это еще не повод неограниченно забрасывать его своими запросами, особенно если это может привести к его отключению - [уголовное наказание](http://sd-company.su/article/security/ddosataka-ugolovnaya-otvetstvennost) никто не отменял. Успешных и безопасных вам исследований!

# Материалы

* [Парсим мемы в python](https://habr.com/ru/company/ods/blog/346632/) - подробная статья на Хабре
* [Продвинутое использование requests](https://2.python-requests.org/en/master/user/advanced/)
* [Репозиторий](https://github.com/DmitrySerg/memology) с исследованием мемов

<img src="pictures/take_all.png" height="200" width="900">  